我们将使用最简单的 Python 和 NumPy 库来复现 Transformer 的核心组件算法。

这里我们不涉及模型的训练、batching 的优化或复杂的框架特性，只关注每个组件的数学运算和逻辑。

我们将主要实现以下组件：

1.  **Positional Encoding (位置编码):** 给序列中的每个位置提供一个唯一的向量表示。
2.  **Scaled Dot-Product Attention (缩放点积注意力):** 注意力的核心计算单元。
3.  **Multi-Head Attention (多头注意力):** 并行运行多个注意力机制，然后合并结果。
4.  **Feed-Forward Network (前馈网络):** 一个简单的两层全连接网络，带激活函数。
5.  **Add & Norm (残差连接与层归一化):** 残差连接防止梯度消失，层归一化稳定训练。
6.  **Encoder Layer (编码器层):** 包含多头注意力和前馈网络，以及残差连接和层归一化。

我们将使用 NumPy 数组来表示张量。形状约定通常是 `(batch_size, sequence_length, model_dim)` 或类似的。

**准备工作:**

In [8]:
import numpy as np
import math

# 为了简化，我们手动实现 softmax
def softmax(x, axis=-1):
    # 提高数值稳定性
    x = x - np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

**1. Positional Encoding (位置编码)**

通过正弦和余弦函数为序列中的每个位置生成一个固定向量。

$$PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d_{model}})$$
$$PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d_{model}})$$

其中，$pos$ 是位置，$i$ 是维度。

In [9]:
def positional_encoding(position, d_model):
    """
    生成位置编码矩阵。

    Args:
        position: 最大序列长度。
        d_model: 模型的维度。

    Returns:
        一个形状为 (1, position, d_model) 的 NumPy 数组。
    """
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / d_model)

    # 对偶数索引应用 sin
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # 对奇数索引应用 cos
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...] # 添加 batch 维度
    return pos_encoding.astype(np.float32)

# --- Test Positional Encoding ---
print("--- Test Positional Encoding ---")
max_seq_len = 50
model_dim = 512
pe = positional_encoding(max_seq_len, model_dim)
print(f"Positional Encoding shape: {pe.shape}") # Expected: (1, 50, 512)
print("-" * 20)

--- Test Positional Encoding ---
Positional Encoding shape: (1, 50, 512)
--------------------


**2. Scaled Dot-Product Attention (缩放点积注意力)**

这是注意力的核心计算：计算 Query 和 Key 的相似度，缩放，通过 softmax 得到注意力权重，然后加权 Value。

$$Attention(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V$$

其中 $d_k$ 是 Key 向量的维度。

In [10]:
def scaled_dot_product_attention(q, k, v, mask=None):
    """
    计算缩放点积注意力。

    Args:
        q: Query 张量，形状 (..., seq_len_q, depth)。
        k: Key 张量，形状 (..., seq_len_k, depth)。
        v: Value 张量，形状 (..., seq_len_v, depth_v)。
            seq_len_k 必须等于 seq_len_v。
        mask: Mask 张量 (用于屏蔽某些连接)，形状 (..., seq_len_q, seq_len_k)。
              通常在解码器中使用 (Look-ahead mask) 或处理变长序列 (Padding mask)。

    Returns:
        output: 注意力计算结果，形状 (..., seq_len_q, depth_v)。
        attention_weights: 注意力权重，形状 (..., seq_len_q, seq_len_k)。
    """
    # 计算 Q 和 K 的点积 (相似度)
    # 形状 (..., seq_len_q, seq_len_k)
    matmul_qk = np.matmul(q, k.transpose(0, 1, 3, 2))

    # 缩放
    d_k = q.shape[-1]
    scaled_attention_logits = matmul_qk / np.sqrt(d_k)

    # 应用 mask (如果提供了)
    if mask is not None:
        # 将 mask 为 0 的位置 (即需要忽略的位置) 的 logits 设置为一个非常小的负数
        scaled_attention_logits = scaled_attention_logits + (mask * -1e9) # 使用 -1e9 代替负无穷

    # softmax 得到注意力权重
    # 形状 (..., seq_len_q, seq_len_k)
    attention_weights = softmax(scaled_attention_logits, axis=-1)

    # 将权重应用于 V
    # 形状 (..., seq_len_q, depth_v)
    output = np.matmul(attention_weights, v)

    return output, attention_weights

# --- Test Scaled Dot-Product Attention ---
print("--- Test Scaled Dot-Product Attention ---")
# 模拟输入 (batch=1, seq_len=3, depth=2)
q = np.random.rand(1, 3, 2)
k = np.random.rand(1, 3, 2)
v = np.random.rand(1, 3, 2)
mask = None # No mask for simplicity

attention_output, attention_weights = scaled_dot_product_attention(q[np.newaxis, ...], # Add head dim for test
                                      k[np.newaxis, ...], # Add head dim for test
                                      v[np.newaxis, ...], # Add head dim for test
                                      mask)
# Remove the added head dim for printing
attention_output = attention_output.squeeze(0)
attention_weights = attention_weights.squeeze(0)

print(f"Q shape: {q.shape}")
print(f"K shape: {k.shape}")
print(f"V shape: {v.shape}")
print(f"Attention Output shape: {attention_output.shape}") # Expected: (1, 3, 2)
print(f"Attention Weights shape: {attention_weights.shape}") # Expected: (1, 3, 3)
print("Sample Attention Weights:\n", attention_weights[0]) # Print weights for the first item in batch
print("-" * 20)

--- Test Scaled Dot-Product Attention ---
Q shape: (1, 3, 2)
K shape: (1, 3, 2)
V shape: (1, 3, 2)
Attention Output shape: (1, 3, 2)
Attention Weights shape: (1, 3, 3)
Sample Attention Weights:
 [[0.25414612 0.39655761 0.34929627]
 [0.29750412 0.36078136 0.34171453]
 [0.24224829 0.40862682 0.34912489]]
--------------------


**3. Multi-Head Attention (多头注意力)**

将 Query, Key, Value 分别通过线性层映射到 `num_heads` 个 "头"，每个头独立计算缩放点积注意力，然后将所有头的输出拼接起来，再通过一个线性层进行最终映射。

In [11]:
def split_heads(x, num_heads, depth):
    """
    将最后一个维度分割成 (num_heads, depth)。
    转置结果以便进行 attention 计算。

    Args:
        x: 输入张量，形状 (batch_size, seq_len, d_model)。
        num_heads: 头数。
        depth: 每个头的维度 (d_model / num_heads)。

    Returns:
        一个形状为 (batch_size, num_heads, seq_len, depth) 的张量。
    """
    # 形状 (batch_size, seq_len, num_heads, depth)
    x = x.reshape(x.shape[0], x.shape[1], num_heads, depth)
    # 形状 (batch_size, num_heads, seq_len, depth)
    return x.transpose(0, 2, 1, 3)

def combine_heads(x):
    """
    合并多头注意力的输出。

    Args:
        x: 输入张量，形状 (batch_size, num_heads, seq_len, depth)。

    Returns:
        一个形状为 (batch_size, seq_len, d_model) 的张量。
    """
    # 形状 (batch_size, seq_len, num_heads, depth)
    x = x.transpose(0, 2, 1, 3)
    # 形状 (batch_size, seq_len, d_model)
    d_model = x.shape[2] * x.shape[3]
    return x.reshape(x.shape[0], x.shape[1], d_model)


def multi_head_attention(q, k, v, mask, d_model, num_heads,
                         Wq, Wk, Wv, Wo):
    """
    实现多头注意力机制。

    Args:
        q: Query 张量，形状 (batch_size, seq_len_q, d_model)。
        k: Key 张量，形状 (batch_size, seq_len_k, d_model)。
        v: Value 张量，形状 (batch_size, seq_len_v, d_model)。
            seq_len_k 必须等于 seq_len_v。
        mask: Mask 张量，形状 (batch_size, 1, 1, seq_len_k) 或 (batch_size, 1, seq_len_q, seq_len_k)。
        d_model: 模型的维度。
        num_heads: 头数。
        Wq, Wk, Wv, Wo: 分别是 Q, K, V 投影和最终输出投影的权重矩阵。
                      Wq, Wk, Wv 形状 (d_model, d_model)。 Wo 形状 (d_model, d_model)。

    Returns:
        output: 多头注意力输出，形状 (batch_size, seq_len_q, d_model)。
        attention_weights: 所有头的注意力权重 (用于可视化或调试)，形状 (batch_size, num_heads, seq_len_q, seq_len_k)。
    """
    depth = d_model // num_heads
    assert d_model % num_heads == 0 # d_model 必须能被 num_heads 整除

    # 1. 通过线性层进行 Q, K, V 的投影
    # 形状 (batch_size, seq_len, d_model)
    q_proj = np.matmul(q, Wq)
    k_proj = np.matmul(k, Wk)
    v_proj = np.matmul(v, Wv)

    # 2. 分割成 num_heads
    # 形状 (batch_size, num_heads, seq_len, depth)
    q_heads = split_heads(q_proj, num_heads, depth)
    k_heads = split_heads(k_proj, num_heads, depth)
    v_heads = split_heads(v_proj, num_heads, depth)

    # 3. 对每个头进行缩放点积注意力
    # attention_output 形状 (batch_size, num_heads, seq_len_q, depth)
    # attention_weights 形状 (batch_size, num_heads, seq_len_q, seq_len_k)
    attention_output, attention_weights = scaled_dot_product_attention(
        q_heads, k_heads, v_heads, mask)

    # 4. 合并所有头的输出
    # 形状 (batch_size, seq_len_q, d_model)
    output_combined = combine_heads(attention_output)

    # 5. 最终的线性投影
    # 形状 (batch_size, seq_len_q, d_model)
    output = np.matmul(output_combined, Wo)

    return output, attention_weights

# --- Test Multi-Head Attention ---
print("--- Test Multi-Head Attention ---")
batch_size = 2
seq_len_q = 4
seq_len_k = 5 # Key/Value sequence can be different length (e.g., encoder output)
d_model = 64
num_heads = 8

# Initialize random weights (in a real model, these would be learned)
# For simplicity, we use d_model for both input and output dimensions of linear layers
Wq_test = np.random.rand(d_model, d_model)
Wk_test = np.random.rand(d_model, d_model)
Wv_test = np.random.rand(d_model, d_model)
Wo_test = np.random.rand(d_model, d_model)

# Simulate input tensors
q_test = np.random.rand(batch_size, seq_len_q, d_model)
k_test = np.random.rand(batch_size, seq_len_k, d_model)
v_test = np.random.rand(batch_size, seq_len_k, d_model)

# Create a simple padding mask (e.g., last token in key/value is padding)
# mask_test = np.zeros((batch_size, 1, 1, seq_len_k))
# mask_test[:, :, :, -1] = 1 # Mask the last position
mask_test = None # No mask for this test

mha_output, mha_weights = multi_head_attention(q_test, k_test, v_test,
                          mask_test, d_model, num_heads,
                          Wq_test, Wk_test, Wv_test, Wo_test)

print(f"Input Q shape: {q_test.shape}")
print(f"Input K shape: {k_test.shape}")
print(f"Input V shape: {v_test.shape}")
print(f"Multi-Head Attention Output shape: {mha_output.shape}") # Expected: (2, 4, 64)
print(f"Multi-Head Attention Weights shape: {mha_weights.shape}") # Expected: (2, 8, 4, 5)
print("-" * 20)

--- Test Multi-Head Attention ---
Input Q shape: (2, 4, 64)
Input K shape: (2, 5, 64)
Input V shape: (2, 5, 64)
Multi-Head Attention Output shape: (2, 4, 64)
Multi-Head Attention Weights shape: (2, 8, 4, 5)
--------------------


**4. Feed-Forward Network (前馈网络)**

一个简单的两层网络，中间使用 ReLU 激活函数。

$$FFN(x) = \max(0, xW_1 + b_1)W_2 + b_2$$

In [12]:
def feed_forward_network(x, W1, b1, W2, b2):
    """
    简单的两层前馈网络。

    Args:
        x: 输入张量，形状 (batch_size, seq_len, d_model)。
        W1: 第一层权重，形状 (d_model, d_ff)。
        b1: 第一层偏置，形状 (d_ff,)。
        W2: 第二层权重，形状 (d_ff, d_model)。
        b2: 第二层偏置，形状 (d_model,)。

    Returns:
        输出张量，形状 (batch_size, seq_len, d_model)。
    """
    # 第一层线性变换 + ReLU
    # 形状 (batch_size, seq_len, d_ff)
    hidden = np.maximum(0, np.matmul(x, W1) + b1)

    # 第二层线性变换
    # 形状 (batch_size, seq_len, d_model)
    output = np.matmul(hidden, W2) + b2

    return output

# --- Test Feed-Forward Network ---
print("--- Test Feed-Forward Network ---")
batch_size = 2
seq_len = 10
d_model = 128
d_ff = 512 # Inner dimension

# Initialize random parameters
W1_test_ffn = np.random.rand(d_model, d_ff)
b1_test_ffn = np.random.rand(d_ff)
W2_test_ffn = np.random.rand(d_ff, d_model)
b2_test_ffn = np.random.rand(d_model)

# Simulate input
input_test_ffn = np.random.rand(batch_size, seq_len, d_model)

ffn_output_test = feed_forward_network(input_test_ffn,
                     W1_test_ffn, b1_test_ffn,
                     W2_test_ffn, b2_test_ffn)

print(f"FFN Input shape: {input_test_ffn.shape}")
print(f"FFN Output shape: {ffn_output_test.shape}") # Expected: (2, 10, 128)
print("-" * 20)

--- Test Feed-Forward Network ---
FFN Input shape: (2, 10, 128)
FFN Output shape: (2, 10, 128)
--------------------


**5. Add & Norm (残差连接与层归一化)**

包含两部分：
1.  **残差连接:** 将子层的输入直接加到其输出上：$x + \text{Sublayer}(x)$。
2.  **层归一化:** 对特征维度进行归一化。

$$LN(x) = \gamma \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$

其中 $\mu$ 和 $\sigma^2$ 是输入 $x$ 在最后一个维度（特征维度）上的均值和方差，$\gamma$ 和 $\beta$ 是可学习的缩放和偏移参数，$\epsilon$ 是为数值稳定性添加的小数。

In [13]:
def layer_norm(x, gamma, beta, epsilon=1e-6):
    """
    执行层归一化。

    Args:
        x: 输入张量。
        gamma: 可学习的缩放参数，形状与 x 的最后一个维度相同。
        beta: 可学习的偏移参数，形状与 x 的最后一个维度相同。
        epsilon: 用于数值稳定性的小值。

    Returns:
        归一化后的张量，形状与 x 相同。
    """
    # 计算最后一个维度上的均值和方差
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)

    # 归一化
    normalized_x = (x - mean) / np.sqrt(var + epsilon)

    # 缩放和偏移
    output = gamma * normalized_x + beta
    return output

def add_and_norm(x, sublayer_output, gamma, beta):
    """
    执行残差连接和层归一化。

    Args:
        x: 子层的输入张量。
        sublayer_output: 子层的输出张量 (MHA 或 FFN)。
        gamma: 层归一化的 gamma 参数。
        beta: 层归一化的 beta 参数。

    Returns:
        经过残差连接和归一化后的张量。
    """
    # 残差连接
    added = x + sublayer_output
    # 层归一化
    normed = layer_norm(added, gamma, beta)
    return normed

# --- Test Add & Norm ---
print("--- Test Add & Norm ---")
batch_size = 2
seq_len = 5
d_model = 32

# Simulate input and sublayer output
input_test_an = np.random.rand(batch_size, seq_len, d_model)
sublayer_output_test_an = np.random.rand(batch_size, seq_len, d_model)

# Initialize random gamma and beta for LayerNorm
gamma_test_an = np.random.rand(d_model)
beta_test_an = np.random.rand(d_model)


add_norm_output_test = add_and_norm(input_test_an, sublayer_output_test_an,
                    gamma_test_an, beta_test_an)

print(f"Input shape for Add & Norm: {input_test_an.shape}")
print(f"Sublayer output shape for Add & Norm: {sublayer_output_test_an.shape}")
print(f"Add & Norm Output shape: {add_norm_output_test.shape}")
# Expected: (2, 5, 32)
# Check if mean and variance of the last dimension are close to 0 and 1 after normalization (before gamma/beta)
# For simplicity, skip detailed numerical check here, rely on shape check.
print("-" * 20)

--- Test Add & Norm ---
Input shape for Add & Norm: (2, 5, 32)
Sublayer output shape for Add & Norm: (2, 5, 32)
Add & Norm Output shape: (2, 5, 32)
--------------------


**6. Encoder Layer (编码器层)**

一个标准的编码器层包含一个多头自注意力模块，接着是 Add & Norm，然后是一个前馈网络，最后再是一个 Add & Norm。

In [14]:
def encoder_layer(x, mask, d_model, num_heads, d_ff,
          # MHA 参数
          Wq_mha, Wk_mha, Wv_mha, Wo_mha, gamma1_ln, beta1_ln,
          # FFN 参数
          W1_ffn, b1_ffn, W2_ffn, b2_ffn, gamma2_ln, beta2_ln):
    """
    实现一个 Transformer 编码器层。

    Args:
        x: 输入张量，形状 (batch_size, seq_len, d_model)。
        mask: 自注意力模块的 mask 张量，形状 (batch_size, 1, seq_len, seq_len)。
        d_model: 模型的维度。
        num_heads: 多头注意力的头数。
        d_ff: 前馈网络的内部维度。
        ... 其他参数为各子层的权重、偏置、gamma、beta。

    Returns:
        输出张量，形状 (batch_size, seq_len, d_model)。
    """

    # Layer 1: Multi-Head Self Attention + Add & Norm
    # 在自注意力中，Q, K, V 都来自同一个输入 x
    attn_output, _ = multi_head_attention(
        x, x, x, mask, d_model, num_heads,
        Wq_mha, Wk_mha, Wv_mha, Wo_mha
    )
    # 残差连接和层归一化
    out1 = add_and_norm(x, attn_output, gamma1_ln, beta1_ln)

    # Layer 2: Feed Forward + Add & Norm
    ffn_output = feed_forward_network(
        out1, W1_ffn, b1_ffn, W2_ffn, b2_ffn
    )
    # 残差连接和层归一化
    out2 = add_and_norm(out1, ffn_output, gamma2_ln, beta2_ln)

    return out2

# # --- Test Encoder Layer ---
# print("--- Test Encoder Layer ---")
# batch_size = 2
# seq_len = 10
# d_model = 128
# num_heads = 8
# d_ff = 512

# # Simulate input
# input_test_el = np.random.rand(batch_size, seq_len, d_model)

# # Create a dummy mask (e.g., for padding or no mask for encoder)
# # In a real encoder, the mask would hide padding tokens.
# # Here, we use None for a simple test (no masking) or create a dummy padding mask.
# # Example padding mask: assume the last 2 tokens are padding in batch 0, and last 1 in batch 1
# mask_test_el = np.zeros((batch_size, 1, seq_len, seq_len))
# # For batch 0, mask positions 8 and 9
# # mask_test_el[0, :, :, 8:] = 1 # This mask is applied to K/V dimensions, so mask columns
# # mask_test_el[0, :, 8:, :] = 1 # If it's self attention, Q dimension is also masked, mask rows

# # A simpler encoder self-attention mask just handles padding.
# # Let's assume batch 0 has seq_len 8, batch 1 has seq_len 9
# seq_lens = [8, 9]
# mask_test_el = np.array([[ [ [0 if i < seq_len else 1 for i in range(seq_len)] for _ in range(seq_len)] for _ in range(1)] for seq_len in seq_lens])
# mask_test_el = mask_test_el[:, np.newaxis, ...] # Add head dimension placeholder (batch, 1, 1, seq_len)

# print(f"Mask shape for Encoder Layer: {mask_test_el.shape}")

# # Initialize ALL parameters for the layer randomly
# Wq_mha_test = np.random.rand(d_model, d_model)
# Wk_mha_test = np.random.rand(d_model, d_model)
# Wv_mha_test = np.random.rand(d_model, d_model)
# Wo_mha_test = np.random.rand(d_model, d_model)
# gamma1_ln_test = np.random.rand(d_model)
# beta1_ln_test = np.random.rand(d_model)

# W1_ffn_test = np.random.rand(d_model, d_ff)
# b1_ffn_test = np.random.rand(d_ff)
# W2_ffn_test = np.random.rand(d_ff, d_model)
# b2_ffn_test = np.random.rand(d_model)
# gamma2_ln_test = np.random.rand(d_model)
# beta2_ln_test = np.random.rand(d_model)

# encoder_output_test = encoder_layer(
#     input_test_el, mask_test_el, d_model, num_heads, d_ff,
#     Wq_mha_test, Wk_mha_test, Wv_mha_test, Wo_mha_test, gamma1_ln_test, beta1_ln_test,
#     W1_ffn_test, b1_ffn_test, W2_ffn_test, b2_ffn_test, gamma2_ln_test, beta2_ln_test
# )

# print(f"Encoder Layer Input shape: {input_test_el.shape}")
# print(f"Encoder Layer Output shape: {encoder_output_test.shape}") # Expected: (2, 10, 128)
# print("-" * 20)

--- Test Encoder Layer ---


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 1) + inhomogeneous part.

In [15]:
# --- Test Encoder Layer ---
print("--- Test Encoder Layer ---")
batch_size = 2
seq_len = 10  # 批次中的最大序列长度 (tensor 的实际长度)
num_heads = 8
d_model = 128
d_ff = 512

# Simulate input
input_test_el = np.random.rand(batch_size, seq_len, d_model)

# Create a dummy mask for padding based on actual sequence lengths
# Assume actual sequence lengths for items in the batch are different
actual_seq_lens = [8, 9] # 第一个序列实际长度为8，第二个为9

# Create a mask of zeros with the target shape (batch, 1, 1, seq_len)
# The 1s are for broadcasting across heads and the query sequence length dimension
mask_test_el = np.zeros((batch_size, 1, 1, seq_len))

# Set mask values to 1 for positions beyond the actual sequence length
for i, current_seq_len in enumerate(actual_seq_lens):
    if current_seq_len < seq_len:
        # 对于当前批次项，从实际长度之后的位置到最大长度都设置为 1
        mask_test_el[i, :, :, current_seq_len:] = 1

# mask_test_el 的形状现在是 (2, 1, 1, 10)，可以正确广播到注意力分数 (2, 8, 10, 10)
print(f"Mask shape for Encoder Layer: {mask_test_el.shape}")
# 打印一个例子看看掩码内容
print("Sample Mask for batch item 0:\n", mask_test_el[0, 0, 0, :])
print("Sample Mask for batch item 1:\n", mask_test_el[1, 0, 0, :])


# Initialize ALL parameters for the layer randomly (unchanged)
Wq_mha_test = np.random.rand(d_model, d_model)
Wk_mha_test = np.random.rand(d_model, d_model)
Wv_mha_test = np.random.rand(d_model, d_model)
Wo_mha_test = np.random.rand(d_model, d_model)
gamma1_ln_test = np.random.rand(d_model)
beta1_ln_test = np.random.rand(d_model)

W1_ffn_test = np.random.rand(d_model, d_ff)
b1_ffn_test = np.random.rand(d_ff)
W2_ffn_test = np.random.rand(d_ff, d_model)
b2_ffn_test = np.random.rand(d_model)
gamma2_ln_test = np.random.rand(d_model)
beta2_ln_test = np.random.rand(d_model)

encoder_output_test = encoder_layer(
    input_test_el, mask_test_el, d_model, num_heads, d_ff,
    Wq_mha_test, Wk_mha_test, Wv_mha_test, Wo_mha_test, gamma1_ln_test, beta1_ln_test,
    W1_ffn_test, b1_ffn_test, W2_ffn_test, b2_ffn_test, gamma2_ln_test, beta2_ln_test
)

print(f"Encoder Layer Input shape: {input_test_el.shape}")
print(f"Encoder Layer Output shape: {encoder_output_test.shape}") # Expected: (2, 10, 128)
print("-" * 20)

--- Test Encoder Layer ---
Mask shape for Encoder Layer: (2, 1, 1, 10)
Sample Mask for batch item 0:
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
Sample Mask for batch item 1:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Encoder Layer Input shape: (2, 10, 128)
Encoder Layer Output shape: (2, 10, 128)
--------------------


**Decoder Components (简要说明)**

解码器层与编码器层类似，但有几个关键区别：

1.  **Masked Multi-Head Self-Attention:** 在解码器的第一个多头自注意力层中，需要使用 *Look-ahead mask* 来防止注意力关注到未来位置的 token，从而保持自回归属性。
2.  **Cross-Attention:** 解码器还有一个多头注意力层，称为交叉注意力。其 Query 来自于前一个解码器层的输出，而 Key 和 Value 来自于 *编码器的输出*。这里通常也需要 Padding mask 来屏蔽编码器输出中的 padding。
3.  解码器层结构：Masked MSA -> Add & Norm -> Cross-Attention -> Add & Norm -> FFN -> Add & Norm。

为了保持代码的简洁性，我们只详细复现了编码器部分的组件，因为它们包含了 Transformer 的核心机制。实现解码器组件会在 `scaled_dot_product_attention` 中使用不同的 mask（Look-ahead mask for self-attention, Padding mask for cross-attention）并调整 `multi_head_attention` 的输入 Q, K, V 的来源即可。

**测试说明:**

上面的代码块中，每个组件实现之后都紧跟着一个简单的测试部分。这些测试：
1.  定义了模拟的输入张量形状和参数。
2.  使用 `np.random.rand` 或 `np.zeros` 初始化随机或零参数。
3.  调用相应的函数。
4.  打印输入和输出的形状，以验证尺寸是否匹配预期。

这些测试主要用于验证算法的维度正确性，而不是验证计算结果的数值正确性（因为参数是随机的）。在一个真正的框架中，会编写更详细的单元测试来检查数值计算是否符合预期。

这个简单的 NumPy 实现展示了 Transformer 各个组件的底层数学运算。在深度学习框架（如 PyTorch, TensorFlow）中，这些操作会被高度优化，并且参数的管理（初始化、更新）由框架自动处理。

In [17]:

def decoder_layer(x, encoder_output, lookahead_mask, padding_mask, d_model, num_heads, d_ff,
                  # Masked MSA parameters
                  Wq_msa, Wk_msa, Wv_msa, Wo_msa, gamma1_ln, beta1_ln,
                  # Cross-Attention parameters
                  Wq_ca, Wk_ca, Wv_ca, Wo_ca, gamma2_ln, beta2_ln,
                  # FFN parameters
                  W1_ffn, b1_ffn, W2_ffn, b2_ffn, gamma3_ln, beta3_ln):
    """
    Implements a Transformer decoder layer.
    """
    # 1. Masked Multi-Head Self-Attention
    masked_msa_output, _ = multi_head_attention(
        x, x, x, lookahead_mask, d_model, num_heads, Wq_msa, Wk_msa, Wv_msa, Wo_msa
    )
    out1 = add_and_norm(x, masked_msa_output, gamma1_ln, beta1_ln)

    # 2. Cross-Attention
    cross_attn_output, _ = multi_head_attention(
        out1, encoder_output, encoder_output, padding_mask, d_model, num_heads,
        Wq_ca, Wk_ca, Wv_ca, Wo_ca
    )
    out2 = add_and_norm(out1, cross_attn_output, gamma2_ln, beta2_ln)

    # 3. Feed Forward Network
    ffn_output = feed_forward_network(out2, W1_ffn, b1_ffn, W2_ffn, b2_ffn)
    out3 = add_and_norm(out2, ffn_output, gamma3_ln, beta3_ln)

    return out3


# --- Test Decoder Layer ---
print("--- Test Decoder Layer ---")
batch_size = 2
seq_len = 10
d_model = 128
num_heads = 8
d_ff = 512

# Simulate inputs
x = np.random.rand(batch_size, seq_len, d_model)
encoder_output = np.random.rand(batch_size, seq_len, d_model)

# Create dummy masks
lookahead_mask = np.zeros((batch_size, 1, seq_len, seq_len))  # Example, needs adjustment
padding_mask = np.zeros((batch_size, 1, 1, seq_len)) # Example, needs adjustment


# Initialize parameters (replace with actual weight initialization)
# ... all weight parameters for MSA, cross-attention, FFN, layer normalization

Wq_msa = np.random.rand(d_model, d_model)
Wk_msa = np.random.rand(d_model, d_model)
Wv_msa = np.random.rand(d_model, d_model)
Wo_msa = np.random.rand(d_model, d_model)
gamma1_ln = np.random.rand(d_model)
beta1_ln = np.random.rand(d_model)

Wq_ca = np.random.rand(d_model, d_model)
Wk_ca = np.random.rand(d_model, d_model)
Wv_ca = np.random.rand(d_model, d_model)
Wo_ca = np.random.rand(d_model, d_model)
gamma2_ln = np.random.rand(d_model)
beta2_ln = np.random.rand(d_model)

W1_ffn = np.random.rand(d_model, d_ff)
b1_ffn = np.random.rand(d_ff)
W2_ffn = np.random.rand(d_ff, d_model)
b2_ffn = np.random.rand(d_model)
gamma3_ln = np.random.rand(d_model)
beta3_ln = np.random.rand(d_model)


decoder_output = decoder_layer(
    x, encoder_output, lookahead_mask, padding_mask, d_model, num_heads, d_ff,
    Wq_msa, Wk_msa, Wv_msa, Wo_msa, gamma1_ln, beta1_ln,
    Wq_ca, Wk_ca, Wv_ca, Wo_ca, gamma2_ln, beta2_ln,
    W1_ffn, b1_ffn, W2_ffn, b2_ffn, gamma3_ln, beta3_ln
)

print(f"Decoder Layer Output shape: {decoder_output.shape}")  # Expected (2,10,128)


--- Test Decoder Layer ---
Decoder Layer Output shape: (2, 10, 128)
